In [ ]:
pip install tensorflow-addons

In [ ]:
import tensorflow as tf  
import tensorflow_datasets as tfds  
import tensorflow_addons as tfa  
%matplotlib inline  
import matplotlib.pyplot as plt  
import numpy as np  
import gc
gc.collect()

In [ ]:
dataset, dataset_info = tfds.load(name='rock_paper_scissors',                                                                                
                                  data_dir='tmp', with_info=True,as_supervised=True)  
dataset_train = dataset['train'] 
dataset_test = dataset['test'] 

In [ ]:
train_size = dataset_info.splits['train'].num_examples  
test_size = dataset_info.splits['test'].num_examples  
dataset_classes = dataset_info.features['label'].num_classes  
print('dataset name:', dataset_info.name)  
print('train dataset:', dataset_train)  
print('test dataset:', dataset_test)  
print('train dataset size:', train_size)  
print('test dataset size:', test_size)  
print('number of classes in train and test dataset:', dataset_classes,
dataset_info.features['label'].names)  
print('shape of images in train and test dataset:', dataset_info.features['image'].shape)

In [ ]:
def plot_image(n=1):
  for image, label in dataset_train.take(n):  
    image = image.numpy()  
    label = label.numpy()  

    image_label = dataset_info.features['label'].int2str(label)  
    plt.imshow(image)  
    plt.title(image_label)  
    plt.colorbar()  

plot_image(5)  
gc.collect()

In [ ]:
def plot_dataset(dataset, num=5):  
  plt.figure(figsize=(15, 15))  
  plot_index = 0  
  for image, label in dataset.take(num):  
    image = image.numpy()  
    label = label.numpy()  

    image_label = dataset_info.features['label'].int2str(label)  

    plot_index+=1  
    plt.subplot(3, 5, plot_index)  
    plt.title(image_label)  
    plt.imshow(image)  

plot_dataset(dataset_train, 5)
gc.collect()

In [ ]:
batch_size = 64  
image_size = 64  

def format_image(image, label):  
  image = tf.cast(image, tf.float32)  
  image = tf.image.resize(image, (image_size, image_size))  
  image /= 255  
  return image, label  

dataset_train = dataset_train.map(format_image)  
dataset_test = dataset_test.map(format_image)  

# Explore preprocessed training dataset images.
plot_dataset(dataset_train) 

In [ ]:
def image_transpose(image):  
  rand = tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32)   
  image = tf.cond(rand < 0.5,   
                  lambda: tf.identity(image),   
                  lambda: tf.image.transpose(image))   
  return image  

def image_flip(image: tf.Tensor) -> tf.Tensor:  
  image = tf.image.random_flip_left_right(image)  
  image = tf.image.random_flip_up_down(image)  
  return image  

def image_rotate(image):  
  image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4,
  dtype=tf.int32))  
  rand = tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32)   
  def random_rotate(image):  
    image = tfa.image.rotate(  
    image, tf.random.uniform(shape=[], minval=0 * np.pi / 180, maxval=360 * np.pi / 180,
    dtype=tf.float32))  
    return image  

  image = tf.cond(rand < 0.5,   
                  lambda: tf.identity(image),   
                  lambda: random_rotate(image))   
  return image    

def image_color(image: tf.Tensor) -> tf.Tensor:  
  image = tf.image.random_saturation(image, lower=0.5, upper=3)  
  image = tf.image.random_brightness(image, max_delta=0.2)  
  image = tf.image.random_contrast(image, lower=0.8, upper=1)  
  image = tf.image.random_hue(image, max_delta=0.03)  
  image = tf.clip_by_value(image, clip_value_min=0, clip_value_max=1)  
  return image

def image_inversion(image: tf.Tensor) -> tf.Tensor:       
  rand = tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32)  
  image = tf.cond(rand < 0.8,   
                  lambda: tf.identity(image),    
                  lambda: tf.math.add(tf.math.multiply(image, -1), 1))   
  return image  

def image_zoom(image: tf.Tensor, min_zoom=0.8, max_zoom=1.0) -> tf.Tensor:       
  image_width, image_height, image_colors = image.shape  
  crop_size = (image_width, image_height)  

  # Generate crop settings, ranging from a 1% to 20% crop.     
  scales = list(np.arange(min_zoom, max_zoom, 0.01))  
  boxes = np.zeros((len(scales), 4))  

  for i, scale in enumerate(scales):           
    x1 = y1 = 0.5 - (0.5 * scale)  
    x2 = y2 = 0.5 + (0.5 * scale)  
    boxes[i] = [x1, y1, x2, y2]  

  def random_crop(img):   # Create different crops for an image         
    crops = tf.image.crop_and_resize(
        [img], boxes=boxes, box_indices=np.zeros(len(scales)),               
        crop_size=crop_size  
        )  
    # Return a random crop
    return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]  
  choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)  

  # Only apply cropping 50% of the time 
  return tf.cond(choice < 0.5, lambda: image, lambda: random_crop(image))  

In [ ]:
def augment_data(image, label):  
  image = image_flip(image)  
  image = image_color(image)  
  image = image_zoom(image)  
  image = image_transpose(image)  
  image = image_inversion(image)  
  image = image_rotate(image)  
  return image, label  

dataset_train_augmented = dataset_train.map(augment_data)  

plot_dataset(dataset_train_augmented)  
gc.collect()

In [ ]:
dataset_train_batches = dataset_train_augmented.shuffle(  
    buffer_size=train_size).batch(batch_size=batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)   
dataset_test_batches = dataset_test.batch(batch_size)  
print(dataset_train_batches)   
print(dataset_test_batches) 

In [ ]:
model = tf.keras.Sequential([  
    tf.keras.layers.Convolution2D(input_shape=(image_size, image_size, 3), filters=64, kernel_size=3, activation='relu'), 
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.Convolution2D(input_shape=(image_size, image_size, 3), filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.Convolution2D(input_shape=(image_size, image_size, 3), filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.Convolution2D(input_shape=(image_size, image_size, 3), filters=128, kernel_size=3, activation='relu'),  
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dropout(0.5),  
    tf.keras.layers.Dense(units=512, activation=tf.keras.activations.relu),  
    tf.keras.layers.Dense(units=dataset_classes, activation=tf.keras.activations.softmax)  
]) 

In [ ]:
model.summary()

In [ ]:
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)  

model.compile(optimizer=rmsprop_optimizer,  
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])  
steps_per_epoch = train_size // batch_size  
validation_steps = test_size // batch_size  

print('steps_per_epoch:', steps_per_epoch)  
print('validation_steps:', validation_steps) 
 
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_accuracy')  

In [ ]:
training_history = model.fit(x=dataset_train_batches.repeat(),  
                             validation_data=dataset_test_batches.repeat(), 
                             epochs=15,   
                             steps_per_epoch=steps_per_epoch,  
                             validation_steps=validation_steps,  
                             callbacks=[early_stopping],  
                             verbose=1) 

In [ ]:
def plot_training_history(training_history):  
    loss = training_history.history['loss']     
    val_loss = training_history.history['val_loss']
    accuracy = training_history.history['accuracy']     
    val_accuracy = training_history.history['val_accuracy']
    plt.figure(figsize=(18, 6))  

    plt.subplot(1, 2, 1)  
    plt.title('Training and Test Loss')  
    plt.xlabel('Epoch')  
    plt.ylabel('Loss')  
    plt.plot(loss, label='Training set')  
    plt.plot(val_loss, label='Test set', linestyle='--')  
    plt.legend()  
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)  
    plt.subplot(1, 2, 2)  
    plt.title('Training and Test Accuracy')  
    plt.xlabel('Epoch')  
    plt.ylabel('Accuracy')  
    plt.plot(accuracy, label='Training set')  
    plt.plot(val_accuracy, label='Test set', linestyle='--')  
    plt.legend()  
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)  
    plt.show()  

plot_training_history(training_history)  
gc.collect()  

In [ ]:
train_loss, train_accuracy = model.evaluate(dataset_train.batch(batch_size).
take(train_size))  
test_loss, test_accuracy = model.evaluate(dataset_test.batch(batch_size).
take(test_size))  

print('Training Loss: ', train_loss)  
print('Training Accuracy: ', train_accuracy)  
print('Test Loss: ', test_loss)  
print('Test Accuracy: ', test_accuracy)

In [ ]:
model.save('RPS_modelV2.h5')

In [1]:
import cv2
import numpy as np
from keras import models
import sys
import time
from PIL import Image#Load the saved model

print(sys.version)  

model = models.load_model('RPS_modelV2.h5')# Start capturing Video through webcam
video = cv2.VideoCapture(0)



while True:
    _, frame = video.read()
    kernel = np.ones((3,3),np.uint8)
     
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)# define range of skin color in HSV
    lower_skin = np.array([0,20,70], dtype=np.uint8)
    upper_skin = np.array([20,255,255], dtype=np.uint8)#extract skin colur image
    mask = cv2.inRange(hsv, lower_skin, upper_skin)#extrapolate the hand to fill dark spots within
    mask = cv2.dilate(mask,kernel,iterations = 4)#blur the image
    mask = cv2.GaussianBlur(mask,(5,5),100)
    mask = cv2.resize(mask,(64,64))
    img_array = np.array(mask)
    #print(img_array.shape)# Changing dimension from 128x128 to 128x128x3
    img_array = np.stack((img_array,)*3, axis=-1)
    #Our keras model used a 4D tensor, (images x height x width x channel)
    #So changing dimension 128x128x3 into 1x128x128x3 
    img_array_ex = np.expand_dims(img_array, axis=0)
    print(img_array_ex.shape)
    #Calling the predict method on model to predict gesture in the frame
    prediction = model.predict(img_array_ex)
    print(prediction)
    #cv2.imshow("Capturing", frame)
    cv2.imshow("Capturing", mask)
    key=cv2.waitKey(1)
    if key == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

2022-10-07 14:52:19.714727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 14:52:19.827688: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 14:52:20.267285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/anders/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-10-07 14:52:20.267348: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7

3.10.6 (main, Aug 10 2022, 11:40:04) [GCC 11.3.0]


2022-10-07 14:52:20.689875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 14:52:20.705024: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 14:52:20.705204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 14:52:20.705627: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

(1, 64, 64, 3)


2022-10-07 14:52:22.350810: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500


1/1 [==============================] - 1s 1s/step


2022-10-07 14:52:22.704682: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-07 14:52:22.851962: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 14ms/step
[[0.0000000e+00 1.0000000e+00 1.4600684e-29]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 15ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 15ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 15ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 19ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 16ms/step
[[0. 0. 1.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 13ms/step
[[0. 1. 0.]]
(1, 64, 64, 3)
1/1 [==============================] - 0s 14ms/step
[[0. 1.

KeyboardInterrupt: 

: 